In [ ]:
#按LTE小区名称进行查询并显示KPI图形
import pymysql.cursors
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
get_ipython().run_line_magic('matplotlib', 'inline')

connection = pymysql.connect(host='localhost',
                             user='Test',
                             password='1234',
                             db='sts',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
if connection:
    print("Connect to database successfully")
    
try:
    with connection.cursor() as cursor:
        # Read a single record
        CELLNAME=input("CellName=")
        CELLNAME=CELLNAME.strip().upper()
        StartDate=input("StartDate(\"YYYY-MM-DD\")=")
        StartDate=StartDate.strip()
        sql =("SELECT * FROM `daily_cell_lte_5.1` WHERE Time>= \'"+StartDate+"\'and CELLNAME=\'"+CELLNAME+"\' ORDER BY Time ASC")
        cursor.execute(sql)
        result = cursor.fetchall()
        if not result:
            print("Query result is empty")
except:
    print("Error occured when querying")
finally:
    connection.close()

KPI=['L-RRC Setup Success Rate (Service)',
     'L-eRAB Setup Success Rate_ALL',
     'L-RRC Drop Rate',
     'L-eRAB Drop Rate_ALL',
     'L-PRB Utilization DL',
     'L-Average UL Interference',
     'L-CSFB Execution Rate']

ERAB_FAIL_CAUSE=['L.E-RAB.FailEst.MME',
                 'L.E-RAB.FailEst.TNL',
                 'L.E-RAB.FailEst.TNL.DLRes',
                 'L.E-RAB.FailEst.TNL.ULRes',
                 'L.E-RAB.FailEst.TNL.Other',
                 'L.E-RAB.FailEst.RNL',
                 'L.E-RAB.FailEst.RNL.Other',
                 'L.E-RAB.FailEst.RNL.eNodeB.NormRel',
                 'L.E-RAB.FailEst.RNL.eNodeB.AbnormRel',
                 'L.E-RAB.FailEst.NoReply',
                 'L.E-RAB.FailEst.Conflict.Hofail',
                 'L.E-RAB.FailEst.SecurModeFail',
                 'L.E-RAB.FailEst.SRBReset',
                 'L.E-RAB.FailEst.Other',
                 'L.E-RAB.FailEst.NoRadioRes']
RRC_DROP_CAUSE=['L.UECNTX.AbnormRel.Act',
                'L.UECNTX.AbnormRel.UlWeak',
                'L.UECNTX.Rel.eNodeB.InitAttEst.MMENoReply',
                'L.UECNTX.Rel.eNodeB.Rnl',
                'L.UECNTX.Rel.eNodeB.HOFailure',
                'L.UECNTX.Rel.eNodeB.UeLost',
                'L.UECNTX.Rel.MME.AbnormRel.Act',
                'L.UECNTX.Rel.S1Reset.eNodeB',
                'L.UECNTX.Rel.S1Reset.MME']

if result:
    df=pd.DataFrame(result)
    Time_range=df['Time'].values.tolist()
    fig_number=0
    for x in KPI:
        fig_number=+1
        plt.figure(fig_number)
        plt.figure(figsize=(12,3.5),facecolor='whitesmoke')
        ax=plt.subplot(111)
        ax.plot(Time_range,df[x],'b-',label=CELLNAME)
        ax.set(title=x)
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))  #设置日期间隔为1天
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax.set_xticks(Time_range)
        ax.spines['top'].set_visible(False)  
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_color('lightgrey')  
        ax.spines['left'].set_color('lightgrey')
        ax.legend(bbox_to_anchor=(1,1.13),loc='upper right')
        if x=='L-Average UL Interference':
            ax.set_ylim((-125,-90))
        plt.xticks(rotation=90)
    
    #ERAB ESTABLISHMENT FAILURE CAUSE   
    fig_number=+1
    plt.figure(fig_number)
    plt.figure(figsize=(12,3.5),facecolor='whitesmoke')
    ax_erab_fail=plt.subplot(111)
    for erab_fail in ERAB_FAIL_CAUSE:
        ax_erab_fail.plot(Time_range,df[erab_fail],'-',label=erab_fail)
    ax_erab_fail.set(title='E-RAB Establishment Fail Cause-All')
    ax_erab_fail.xaxis.set_major_locator(mdates.DayLocator(interval=1))  #设置日期间隔为1天
    ax_erab_fail.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax_erab_fail.set_xticks(Time_range)   #设置X轴日期显示范围
    ax_erab_fail.spines['top'].set_visible(False)  
    ax_erab_fail.spines['right'].set_visible(False)
    ax_erab_fail.spines['bottom'].set_color('lightgrey')  
    ax_erab_fail.spines['left'].set_color('lightgrey')  
    ax_erab_fail.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.xticks(rotation=90)
    
    #RRC DROP CAUSE   
    fig_number=+1
    plt.figure(fig_number)
    plt.figure(figsize=(12,3.5),facecolor='whitesmoke')
    ax_rrc_drop=plt.subplot(111)
    for rrc_drop_cause in RRC_DROP_CAUSE:
        ax_rrc_drop.plot(Time_range,df[rrc_drop_cause],'-',label=rrc_drop_cause)
    ax_rrc_drop.set(title='UE Context Release Cause-All')
    ax_rrc_drop.xaxis.set_major_locator(mdates.DayLocator(interval=1))  #设置日期间隔为1天
    ax_rrc_drop.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    ax_rrc_drop.set_xticks(Time_range)   #设置X轴日期显示范围
    ax_rrc_drop.spines['top'].set_visible(False)  
    ax_rrc_drop.spines['right'].set_visible(False)
    ax_rrc_drop.spines['bottom'].set_color('lightgrey')  
    ax_rrc_drop.spines['left'].set_color('lightgrey')  
    ax_rrc_drop.legend(bbox_to_anchor=(1.02, 1), loc=2, borderaxespad=0.)
    plt.xticks(rotation=90)
    
plt.show()

In [ ]:
#ERAB FAIL CAUSE统计
df_erab_fail_cause=df.loc[0:,ERAB_FAIL_CAUSE]
df_erab_fail_cause['Time']=Time_range
df_erab_fail_cause=df_erab_fail_cause.set_index('Time')
df_erab_fail_cause=df_erab_fail_cause.T
df_erab_fail_cause=df_erab_fail_cause.astype(int)
df_erab_fail_cause

In [ ]:
#RRC DROP CAUSE统计
df_rrc_drop_cause=df.loc[0:,RRC_DROP_CAUSE]
df_rrc_drop_cause['Time']=Time_range
df_rrc_drop_cause=df_rrc_drop_cause.set_index('Time')
df_rrc_drop_cause=df_rrc_drop_cause.T
df_rrc_drop_cause=df_rrc_drop_cause.astype(int)
df_rrc_drop_cause

In [ ]:
#CSFB执行失败次数统计

#从df中提取计算失败次数相关的几列
df_csfb_exec=df.loc[0:,['Time','CELLNAME','L-CSFB Execution Times','L.CSFB.PrepSucc']]

#方法1：
#使用DataFrame apply和lambda生成失败率（1-成功率）
#df_csfb_exec=df.loc[0:,['L-CSFB Execution Rate']]
#df_csfb_exec_fail=df.loc[0:,'L-CSFB Execution Rate'].apply(lambda x:100-x) 
#df_fail_num=df_csfb_exec*df_csfb_exec_fail/100
#csfb_fail_num=df_fail_num.values.tolist()
#df_csfb_fail_num=pd.DataFrame(csfb_fail_num,index=Time_range,columns=['CSFB_FAIL_NUM']).astype(int)

#方法2
#生成列‘CSFB Exec Fail Times’的数据
df_csfb_exec_fail=df['L.CSFB.PrepSucc']-df['L-CSFB Execution Times']
csfb_exec_fail_number=df_csfb_exec_fail.values.tolist()

#增加一列‘CSFB Exec Fail Times’
df_csfb_exec['CSFB Exec Fail Times']=csfb_exec_fail_number

#设置Time列为index
df_csfb_exec=df_csfb_exec.set_index('Time')

#选取‘CSFB Exec Fail Times’列plot
df_csfb_exec['CSFB Exec Fail Times'].plot(figsize=(9,3),title='CSFB_FAIL_NUM')